In [42]:
import cv2
import numpy as np 
import os
import pandas as pd
from PIL import Image
from skimage.io import imread
from skimage.transform import resize


In [67]:
input_dir = 'C:/Users/liang/OneDrive/Desktop/Maplestory-Classifier' 








In [68]:
data = []
categories = ['Mushmom_Classification'] 


In [73]:
for category in categories: 
    for file in os.listdir(os.path.join(input_dir,category)):   
        img_path = os.path.join(input_dir,category,file)
        img = imread(img_path)
        img = resize(img,(15,15))     
        print(img)
       



[[[0.99607843 1.         0.97647059]
  [0.99607843 1.         0.97647059]
  [0.99607843 1.         0.97647059]
  [0.99607843 1.         0.97647059]
  [0.99607824 0.99999987 0.97647021]
  [0.99586026 0.99977513 0.97623789]
  [0.98588483 0.98921771 0.96485537]
  [0.95201298 0.95068132 0.91876027]
  [0.95317807 0.94947639 0.91703144]
  [0.9865204  0.98931582 0.96470244]
  [0.99582812 0.99973995 0.97618583]
  [0.99607806 0.99999967 0.97647024]
  [0.99607843 1.         0.97647059]
  [0.99607843 1.         0.97647059]
  [0.99607843 1.         0.97647059]]

 [[0.99607843 1.         0.97647059]
  [0.99607843 1.         0.97647059]
  [0.99607843 1.         0.97647059]
  [0.99607837 0.99999989 0.97647044]
  [0.99597791 0.99990144 0.9762881 ]
  [0.98250771 0.98447677 0.95901276]
  [0.86001318 0.82518612 0.75285045]
  [0.82674663 0.72182532 0.51273847]
  [0.83694372 0.71257802 0.49298122]
  [0.8685547  0.84221338 0.75689411]
  [0.98071881 0.98306967 0.95717276]
  [0.99597996 0.99990304 0.97637257]

<h1 style="font-size:24px; font-family: Arial, sans-serif;">  Converting Image_To_Array </h1>

<h1 style="font-size:24px; font-family: Arial, sans-serif;"> Images </h1>

<h1 style="font-size:24px; font-family: Arial, sans-serif;"> --->  Stuck HERE  <--- </h1>
